In [11]:
import os
import pandas as pd
from torch.utils.data import Dataset, DataLoader
from sklearn.model_selection import train_test_split
from torchvision import transforms
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from PIL import Image
import matplotlib.pyplot as plt
from skimage.io import imread
from skimage.color import rgb2gray
from skimage.feature import canny
import numpy as np
from gtda.homology import VietorisRipsPersistence
from gtda.diagrams import PersistenceEntropy

In [12]:
human_data1 = pd.read_csv("../datasets/human/list_attr_celeba.csv")
human_data2 = pd.read_csv("../datasets/human/list_bbox_celeba.csv")
human_data3 = pd.read_csv("../datasets/human/list_eval_partition.csv")
human_data4 = pd.read_csv("../datasets/human/list_landmarks_align_celeba.csv")

# Display the columns in human datasets
print("Human dataset1: ", human_data1.columns)
print("\nHuman dataset2: ", human_data2.columns)
print("\nHuman dataset3: ", human_data3.columns)
print("\nHuman dataset4: ", human_data4.columns)

# Load AI dataset
ai_datasets = pd.read_json("../datasets/ai/ffhq-dataset-v1-processed.json")
print("\nAI dataset: ", ai_datasets.columns)

# Merging human datasets
merged_df = pd.merge(human_data1, human_data2, on='image_id').merge(human_data3, on='image_id').merge(human_data4, on='image_id')
print("\nMerged dataset: ", merged_df.head())

# Label human data as 0 (real human)
merged_df["label"] = 0

# Prepare AI data
ai_df = pd.DataFrame({
    "image_id" : [f"{i}.png" for i in range(len(ai_datasets))],
    "label" : 1 # 1 for AI generated
})

# Combine human and AI dataframes
human_df = merged_df[["image_id", "label"]]
combined_df = pd.concat([human_df, ai_df], ignore_index=True)
combined_df = combined_df.sample(frac=1).reset_index(drop=True)


Human dataset1:  Index(['image_id', '5_o_Clock_Shadow', 'Arched_Eyebrows', 'Attractive',
       'Bags_Under_Eyes', 'Bald', 'Bangs', 'Big_Lips', 'Big_Nose',
       'Black_Hair', 'Blond_Hair', 'Blurry', 'Brown_Hair', 'Bushy_Eyebrows',
       'Chubby', 'Double_Chin', 'Eyeglasses', 'Goatee', 'Gray_Hair',
       'Heavy_Makeup', 'High_Cheekbones', 'Male', 'Mouth_Slightly_Open',
       'Mustache', 'Narrow_Eyes', 'No_Beard', 'Oval_Face', 'Pale_Skin',
       'Pointy_Nose', 'Receding_Hairline', 'Rosy_Cheeks', 'Sideburns',
       'Smiling', 'Straight_Hair', 'Wavy_Hair', 'Wearing_Earrings',
       'Wearing_Hat', 'Wearing_Lipstick', 'Wearing_Necklace',
       'Wearing_Necktie', 'Young'],
      dtype='object')

Human dataset2:  Index(['image_id', 'x_1', 'y_1', 'width', 'height'], dtype='object')

Human dataset3:  Index(['image_id', 'partition'], dtype='object')

Human dataset4:  Index(['image_id', 'lefteye_x', 'lefteye_y', 'righteye_x', 'righteye_y',
       'nose_x', 'nose_y', 'leftmouth_x', 'leftmo

In [13]:
transform = transforms.Compose([
    transforms.Resize((64, 64)),
    transforms.ToTensor(),
])

In [14]:
class ImageDataset(Dataset):
    def __init__(self, dataframe, transform=None, root_dir="../datasets"):
        self.dataframe = dataframe
        self.transform = transform
        self.root_dir = root_dir

    def __len__(self):
        return len(self.dataframe)

    def __getitem__(self, idx):
        row = self.dataframe.iloc[idx]
        img_name = row["image_id"]
        label = row["label"]

        if label == 0:
            img_path = os.path.join(self.root_dir, "human", "img_align_celeba", img_name)
        else:
            img_path = os.path.join(self.root_dir, "ai", "faces_dataset", img_name)
        
        image = Image.open(img_path).convert("RGB")
        if self.transform:
            image = self.transform(image)

        return image, label, img_path


In [15]:
train_df, val_df = train_test_split(combined_df, test_size=0.2, stratify=combined_df['label'], random_state=42)

In [16]:
train_dataset = ImageDataset(train_df, transform=transform)
val_dataset = ImageDataset(val_df, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=2)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False, num_workers=2)

In [17]:
class SimpleCNN(nn.Module):
    def __init__(self):
        super(SimpleCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 16, 3, padding=1)  # 3 channels for RGB, 16 filters for 1st conv layer
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1)  # 32 filters for 2nd conv layer
        self.pool = nn.MaxPool2d(2, 2)  # 2x2 pooling
        self.fc1 = nn.Linear(32 * 16 * 16, 128)  # 32 channels, 16x16 feature map after pooling
        self.fc2 = nn.Linear(128 + 10, 1)  # 10 for TDA features, 1 output for binary classification

    def forward(self, x, tda_features):
        x = self.pool(F.relu(self.conv1(x)))
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 32 * 16 * 16)  # Flatten the feature map
        x = F.relu(self.fc1(x))
        x = torch.cat((x, tda_features), dim=1)  # Combine CNN and TDA features
        x = torch.sigmoid(self.fc2(x))  # Sigmoid activation for binary classification
        return x


In [18]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = SimpleCNN().to(device)

criterion = nn.BCELoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)


In [19]:
def train(model, loader, optimizer, criterion):
    model.train()
    total_loss = 0
    for images, labels in loader:
        images = images.to(device)
        labels = labels.float().unsqueeze(1).to(device)
        tda_vecs = extract_tda_features(paths)  # Placeholder for TDA feature extraction
        
        optimizer.zero_grad()
        outputs = model(images, tda_vecs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        total_loss += loss.item()
    return total_loss / len(loader)

def validate(model, loader, criterion):
    model.eval()
    total_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device)
            labels = labels.float().unsqueeze(1).to(device)
            outputs = model(images)
            loss = criterion(outputs, labels)
            total_loss += loss.item()

            predicted = (outputs > 0.5).float()
            correct += (predicted == labels).sum().item()
            total += labels.size(0)
    accuracy = correct / total
    return total_loss / len(loader), accuracy


In [ ]:
num_epochs = 5
for epoch in range(num_epochs):
    train_loss = train(model, train_loader, optimizer, criterion)
    val_loss, val_acc = validate(model, val_loader, criterion)


In [ ]:
train_losses = []
val_losses = []
val_accuracies = []

for epoch in range(num_epochs):
    train_loss = train(model, train_loader, optimizer, criterion)
    val_loss, val_accuracy = validate(model, val_loader, criterion)

    train_losses.append(train_loss)
    val_losses.append(val_loss)
    val_accuracies.append(val_accuracy)

    print(f"Epoch [{epoch+1}/{num_epochs}], Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Val Accuracy: {val_accuracy*100:.2f}%")

# Plot Loss Curves
plt.figure(figsize=(12,5))

plt.subplot(1,2,1)
plt.plot(train_losses, label="Train Loss")
plt.plot(val_losses, label="Validation Loss")
plt.xlabel("Epochs")
plt.ylabel("Loss")
plt.title("Loss over Epochs")
plt.legend()

# Plot Accuracy Curve
plt.subplot(1,2,2)
plt.plot(val_accuracies, label="Validation Accuracy", color='green')
plt.xlabel("Epochs")
plt.ylabel("Accuracy")
plt.title("Validation Accuracy over Epochs")
plt.legend()

plt.show()


In [ ]:
torch.save(model.state_dict(), "simple_cnn_real_vs_ai.pth")
print("Model saved successfully!")


In [ ]:
def predict_image(image_path, model, transform):
    model.eval()
    image = Image.open(image_path).convert('RGB')
    image = transform(image).unsqueeze(0).to(device)

    with torch.no_grad():
        output = model(image)
        prediction = (output > 0.5).float()

    if prediction.item() == 0:
        print(f"Prediction: Real Human Image 🧑‍🦰")
    else:
        print(f"Prediction: AI Generated Image 🤖")


In [ ]:
def image_to_point_cloud(image_path):
    image = imread(image_path)
    gray = rgb2gray(image)
    edges = canny(gray) 
    points = np.column_stack(np.nonzero(edges))
    return points

VR = VietorisRipsPersistence(homology_dimensions=[0, 1])
PE = PersistenceEntropy()

def get_tda_vector(image_path):
    gray = rgb2gray(imread(image_path)) 
    edges = canny(gray) 
    points = np.column_stack(np.nonzero(edges)).reshape(1, -1, 2)
    diag = VR.fit_transform(points)
    vec = PE.fit_transform(diag)
    return vec.squeeze()

def extract_tda_features(batch_paths):
    vectors = [get_tda_vector(path) for path in batch_paths]
    return torch.tensor(vectors, dtype=torch.float32).to(device)


In [ ]:
def validate(model, loader, criterion):
    model.eval() # Set model to evaluation mode
    total_loss = 0
    correct = 0
    total = 0
    with torch.no_grad():
        for images, labels in loader:
            images = images.to(device) # Move images to GPU if available
            labels = labels.float().unsqueeze(1).to(device) # Unsqueeze to add a dimension for binary classification
            outputs = model(images) # Forward pass through the model
            loss = criterion(outputs, labels) # Calculate loss
            total_loss += loss.item() # Accumulate loss

            predicted = (outputs > 0.5).float() # Convert probabilities to binary predictions
            correct += (predicted == labels).sum().item() # Count correct predictions
            total += labels.size(0) 
    accuracy = correct / total
    return total_loss / len(loader), accuracy